In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from zipfile import ZipFile
from urllib.request import urlretrieve

# Import the required libraries for file upload
from ipywidgets import FileUpload
import io

%matplotlib inline

In [ ]:
# Create file upload widgets for both images
ref_upload_widget = FileUpload(accept='.jpg', description='Upload Reference Form')
scanned_upload_widget = FileUpload(accept='.jpg', description='Upload Scanned Form')

# Display the file upload widgets
display(ref_upload_widget, scanned_upload_widget)

In [ ]:
# Define a function to read the uploaded files
def read_uploaded_images(ref_uploaded_files, scanned_uploaded_files):
    # Check if both files are uploaded
    if len(ref_uploaded_files) != 1 or len(scanned_uploaded_files) != 1:
        print("Please upload one reference form and one scanned form.")
        return None, None
    
    # Unpack the tuple to get file data
    ref_uploaded_file = ref_uploaded_files[0]
    scanned_uploaded_file = scanned_uploaded_files[0]
    
    # Read the reference form image
    ref_filename = ref_uploaded_file.name
    ref_file_data = ref_uploaded_file['content']
    ref_nparr = np.frombuffer(ref_file_data, np.uint8)
    ref_image = cv2.imdecode(ref_nparr, cv2.IMREAD_COLOR)
    ref_image = cv2.cvtColor(ref_image, cv2.COLOR_BGR2RGB)
    
    # Read the scanned form image
    scanned_filename = scanned_uploaded_file.name
    scanned_file_data = scanned_uploaded_file['content']
    scanned_nparr = np.frombuffer(scanned_file_data, np.uint8)
    scanned_image = cv2.imdecode(scanned_nparr, cv2.IMREAD_COLOR)
    scanned_image = cv2.cvtColor(scanned_image, cv2.COLOR_BGR2RGB)
    
    return ref_image, scanned_image

In [ ]:
# Define a function to process and display images
def process_images(_):
    # Get the uploaded file data for the reference form and scanned form
    ref_uploaded_files = ref_upload_widget.value
    scanned_uploaded_files = scanned_upload_widget.value
    
    # Read the uploaded images
    ref_image, scanned_image = read_uploaded_images(ref_uploaded_files, scanned_uploaded_files)
    
    if ref_image is not None and scanned_image is not None:
        # Convert images to grayscale
        ref_gray = cv2.cvtColor(ref_image, cv2.COLOR_RGB2GRAY)
        scanned_gray = cv2.cvtColor(scanned_image, cv2.COLOR_RGB2GRAY)
        
        # Detect ORB features and compute descriptors with an increased number of features
        MAX_NUM_FEATURES = 10000  # Increase the number of features
        orb = cv2.ORB_create(MAX_NUM_FEATURES)
        keypoints1, descriptors1 = orb.detectAndCompute(ref_gray, None)
        keypoints2, descriptors2 = orb.detectAndCompute(scanned_gray, None)
        
        # Match features
        matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
        matches = list(matcher.match(descriptors1, descriptors2, None))
        
        # Sort matches by score
        matches.sort(key=lambda x: x.distance, reverse=False)
        
        # Remove not-so-good matches
        numGoodMatches = int(len(matches) * 0.5)
        matches = matches[:numGoodMatches]
        
        # Calculate and display the percentage and number of feature matches
        total_matches = len(matches)
        total_keypoints1 = len(keypoints1)
        total_keypoints2 = len(keypoints2)
        match_percentage = (total_matches / min(total_keypoints1, total_keypoints2)) * 100
        
        print(f"Number of feature matches: {total_matches}")
        print(f"Percentage of feature matches: {match_percentage:.2f}%")
        
        # Extract location of good matches
        points1 = np.zeros((len(matches), 2), dtype=np.float32)
        points2 = np.zeros((len(matches), 2), dtype=np.float32)
        
        for i, match in enumerate(matches):
            points1[i, :] = keypoints1[match.queryIdx].pt
            points2[i, :] = keypoints2[match.trainIdx].pt
        
        # Find homography
        h, mask = cv2.findHomography(points2, points1, cv2.RANSAC)
        
        # Use homography to warp image
        height, width, channels = ref_image.shape
        scanned_image_aligned = cv2.warpPerspective(scanned_image, h, (width, height))
        
        # Display the original form and scanned form images
        plt.figure(figsize=[20, 10])
        plt.subplot(131)
        plt.imshow(ref_image)
        plt.axis("off")
        plt.title("Reference Form (Uploaded)")
        
        plt.subplot(132)
        plt.imshow(scanned_image)
        plt.axis("off")
        plt.title("Scanned Form (Uploaded)")
        
        plt.subplot(133)
        plt.imshow(scanned_image_aligned)
        plt.axis("off")
        plt.title("Aligned Scanned Form")

In [ ]:
# Process uploaded images when a button is clicked
from ipywidgets import Button

process_button = Button(description="Process Images")

# Attach the process_images function to the button's click event
process_button.on_click(process_images)

# Display the process button
display(process_button)